<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#What-Does-the-CASE-Statement-Do?" data-toc-modified-id="What-Does-the-CASE-Statement-Do?-1"><span class="toc-item-num">1&nbsp;&nbsp;</span><font color="red">What Does the <code>CASE</code> Statement Do?</font></a></span><ul class="toc-item"><li><span><a href="#Case-Statements" data-toc-modified-id="Case-Statements-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Case Statements</a></span></li></ul></li><li><span><a href="#So-What-Can-I-Do-with-a-Case-Statement?" data-toc-modified-id="So-What-Can-I-Do-with-a-Case-Statement?-2"><span class="toc-item-num">2&nbsp;&nbsp;</span><font color="orange">So What Can I Do with a Case Statement?</font></a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Create-Categories-to-Classify-Values" data-toc-modified-id="Create-Categories-to-Classify-Values-2.0.1"><span class="toc-item-num">2.0.1&nbsp;&nbsp;</span>Create Categories to Classify Values</a></span></li><li><span><a href="#Normalize-Values-in-Your-Dataset" data-toc-modified-id="Normalize-Values-in-Your-Dataset-2.0.2"><span class="toc-item-num">2.0.2&nbsp;&nbsp;</span>Normalize Values in Your Dataset</a></span></li><li><span><a href="#Create-a-Boolean-Field" data-toc-modified-id="Create-a-Boolean-Field-2.0.3"><span class="toc-item-num">2.0.3&nbsp;&nbsp;</span>Create a Boolean Field</a></span></li></ul></li></ul></li></ul></div>

# SQL Extras

## <font color=red>What Does the `CASE` Statement Do?</font>

### Case Statements

**The SQL `CASE` Statement** is a conditional expression, which allows me to create a new temporary column in mu result set based on 'If this, then...' logic. A `CASE`statement is added to the `SELECT` statement, after all of the existing fields I have selected to display; it begins with `CASE` and ends with `END AS new_field_name`.

<div class="alert alert-block alert-warning">It's helpful to include an `ELSE` clause to catch any possible values that fall outside of your conditional clauses because otherwise, if those values exist, they will be returned as `NULL`.</div>

## <font color=orange>So What Can I Do with a Case Statement?</font>

#### Create Categories to Classify Values

Using `CASE` comes in very handy when I want to create categories from existing data based on my dataset using conditionals, but I don't have database permissions to create a new table.

>For example, if I have a field called `high_temp` containing integers that represent the high temperature recorded for a particular day, I can add a `CASE` statement to my query that will create a new field called `weather` classifying values in `high_temp` as one of three categorical variables, `cold`, `warm`, `hot`, based on temperature ranges that I set as conditionals.

I can create a column of categorical variables from a numeric column. I can even add a `GROUP BY` and a `COUNT(*)` to find out which category in the `weather` field has the highest count.

```sql
USE us_weather_db;
SELECT high_temp,
        COUNT(*) AS days,
    CASE high_temp > 90 THEN 'hot'
        WHEN high_temp BETWEEN 70 AND 90 THEN 'warm'
        WHEN high_temp < 70 THEN 'cold'
        ELSE 'idk'
    END AS 'weather'
FROM us_weather
GROUP BY high_temp
ORDER BY days DESC;
```
My query can have conditionals that use values from more than one field to create a new column. By throwing in a `COUNT(*) AS number_customers`, I have a useful result set for making comparisons.

```sql
USE telco_churn;
SELECT internet_service_type_id,
		device_protection,
		COUNT(*) AS number_customers,
	CASE WHEN internet_service_type_id = 1 AND device_protection = 'Yes' THEN 'DSL_Protection'
		WHEN internet_service_type_id = 1 AND device_protection = 'No' THEN 'DSL_Unprotected'
		WHEN internet_service_type_id = 2 AND device_protection = 'Yes' THEN 'Fiber_Optic_Protected'
		WHEN internet_service_type_id = 2 AND device_protection = 'No' THEN 'Fiber_Optic_Unprotected'
		WHEN internet_service_type_id = 3 THEN 'No_Internet_Service'
		ELSE internet_service_type_id
	END AS internet_types
FROM customers
GROUP BY internet_service_type_id, device_protection
ORDER BY number_customers;
```

#### Normalize Values in Your Dataset

Sometimes my dataset may be messy (hahaha, sometimes), and I will need to normalize the values. `CASE` can come in handy here, as well.

```sql
SELECT title,
       salary,
       gender,
    CASE gender
        WHEN 'M' THEN 'male'
        WHEN '0' THEN 'male'
        WHEN 'F' THEN 'female'
        WHEN '1' THEN 'female'
        ELSE 'neither'
    END AS gender_norm
FROM employees
```

#### Create a Boolean Field

Sometimes it's easier to work with a boolean value, and `CASE` can be helpful here, as well.

```sql
USE titanic_db;
SELECT *,
	CASE sex 
		WHEN 'female' THEN 1
		ELSE 0
	END AS is_female
FROM passengers; 
```